In [1]:
import praw, json, os, statistics
from datetime import datetime
import pandas as pd

In [2]:
user_agent = 'python:sentiment_analysis:v0.0.1 (by u/Doomsday39)'
reddit = praw.Reddit("bot", user_agent=user_agent)
print(reddit.read_only)

True


In [3]:
smoking_words = ['smoke', 'smoker', 'smoking', 'cigar', 'cigarette', 'weed', 'marijuana', 'drug', 'juul', 'cannabis', 'tobacco', 'opiate', 'vape', 'vaping', 'addiction']
subreddits = [reddit.subreddit('stopsmoking'), reddit.subreddit('quittingsmoking'), reddit.subreddit('QuitVaping'), reddit.subreddit('QuittingJUUL')]
times = ['all', 'hour', 'day', 'week', 'month', 'year']

In [4]:
subs = []
for word in smoking_words:
    for sub in reddit.subreddits.search(word):
        if sub not in subs:
            subs.append(sub)
            print(sub.display_name)

smoking
Cigarettes
trees
BBQ
PopSmoke
hiphopheads
DMT
cigars
stopsmoking
GlobalOffensive
weed
PipeTobacco
Rainbow6
hookah
pics
electronic_cigarette
leaves
uktrees
saplings
AskReddit
Drugs
Showerthoughts
entwives
teenagers
Marijuana
todayilearned
mildlyinteresting
memes
StonerEngineering
see
funny
unpopularopinion
Vaping
MortalKombat
vaporents
dankmemes
circlejerk
WorldOfWarships
worldnews
csgo
Warthunder
interestingasfuck
woahdude
cubancigars
OldSchoolCool
oddlysatisfying
NoStupidQuestions
australia
videos
apexlegends
Knife_Swap
WTF
Petioles
pelletgrills
microgrowery
AdviceAnimals
politics
drugscirclejerk
Jokes
Portland
FLMedicalTrees
highdeas
Traeger
ShittyLifeProTips
ArtOfRolling
BattlefieldV
hempflowers
gifs
IndianEnts
news
Simulated
eldertrees
imaginedragons
blender
VALORANT
quittingsmoking
AmItheAsshole
trashy
reddit.com
WritingPrompts
Stims
explainlikeimfive
SiegeAcademy
FortNiteBR
opiates
ausents
oilpen
bayarea
2007scape
conspiracy
knifeclub
food
DotA2
Crainn
LSD
CallOfDutyMobil

In [5]:
all_data = []
submissions = {}
total_num_of_words = 0
num_of_posts_title_body = 0
num_of_posts_title_url = 0
num_of_posts_per_subreddit = {}
num_of_words_per_subreddit = {}
num_of_words_per_document = []
for subreddit in subreddits:
    subreddit_name = subreddit.display_name
    print(subreddit_name + ' processing...')
    num_of_posts_per_subreddit[subreddit_name] = 0
    submissions[subreddit_name] = []
    i = 0
    while num_of_posts_per_subreddit[subreddit_name] < 1000 and i < len(times):
        for submission in subreddit.top(times[i], limit=1000):
            if submission.id not in submissions[subreddit.display_name]:
                if submission.is_self:
                    submissions[subreddit_name].append(submission.id)
                    post = {
                        'subreddit': subreddit_name,
                        'id': submission.id,
                        'title': submission.title,
                        'url': submission.url,
                        'date_created': datetime.fromtimestamp(submission.created_utc).strftime("%m/%d/%Y, %H:%M:%S"),
                        'body': submission.selftext
                    }
                    num_of_words = len(post['body'].split())
                    total_num_of_words += num_of_words
                    if not post['body'] and post['url'] and post['title']:
                        num_of_posts_title_url += 1
                    elif post['body'] and post['title']:
                        num_of_posts_title_body += 1
                    num_of_posts_per_subreddit[subreddit_name] += 1
                    num_of_posts_per_subreddit[subreddit_name] += num_of_words
                    num_of_words_per_document.append(num_of_words)
                    all_data.append(post)
                    if num_of_posts_per_subreddit[subreddit_name] == 1000:
                        break
        i += 1

stopsmoking processing...
quittingsmoking processing...
QuitVaping processing...
QuittingJUUL processing...


In [6]:
print(len(all_data))
print(num_of_words)
print(statistics.mean(num_of_words_per_document))
print(statistics.stdev(num_of_words_per_document))
print(num_of_posts_title_body)
print(num_of_posts_title_url)
for subreddit in num_of_posts_per_subreddit:
    print('{} has {} posts'.format(subreddit, num_of_posts_per_subreddit[subreddit]))

2541
44
146.71192443919716
190.2266893938697
2410
131
stopsmoking has 35466 posts
quittingsmoking has 74315 posts
QuitVaping has 132188 posts
QuittingJUUL has 133367 posts


In [7]:
output_dir = './data/reddit/'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
with open(output_dir+'reddit_data.json', 'w') as output:
    json.dump(all_data, output)
df = pd.DataFrame.from_dict(all_data)
df.to_csv(output_dir+'reddit_data.csv', index=False)